In [15]:
import numpy as np
import pandas as py
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [16]:
data = py.read_csv('train (3).csv')
testData = py.read_csv('test (2).csv')

In [17]:
#process Title
titleMapping = {"Capt":       "Officer",
                "Col":        "Officer",
                "Major":      "Officer",
                "Jonkheer":   "Royalty",
                "Don":        "Royalty",
                "Sir" :       "Royalty",
                "Dr":         "Officer",
                "Rev":        "Officer",
                "the Countess":   "Royalty",
                "Don":       "Royalty",
                "Dona":       "Royalty",
                "Mme":        "Mrs",
                "Mlle":       "Miss",
                "Ms":         "Mrs",
                "Mr" :        "Mr",
                "Mrs" :       "Mrs",
                "Miss" :      "Miss",
                "Master" :    "Master",
                "Lady" :      "Royalty"}
titlecodeMapping = {"Mr" : 0,"Miss" : 1, "Mrs":2,"Royalty" : 3,"Master" : 4, "Officer" : 5}
def processTitle(x):
    x['Title'] = x['Name'].str.extract(', ([a-zA-Z\s]+)\. ',expand=False).map(titleMapping).map(titlecodeMapping)


In [18]:
#processNameLength
def processNameLength(x):
    x['nameLen'] = x['Name'].apply(lambda x: len(x))
    x['FamilySize'] = x['SibSp'] + x['Parch']+1
    x.drop(['SibSp','Parch','Name'],axis=1,inplace=True)

In [19]:
#processCabin
def processCabin(x):
    x['Cabin'] = x['Cabin'].fillna('U')
    x['Cabin'] = x['Cabin'].apply(lambda x: x[0])
    cabin = py.get_dummies(x['Cabin'],drop_first=True)
    x.drop(['Cabin'],axis=1,inplace=True)
    x = py.concat([x,cabin],axis=1)
    if 'T' not in x.columns:
        x['T']=py.DataFrame(np.zeros(x.shape[0]))

In [20]:
#processAge
def processMissingAge(x):
    if x['Sex'] == 'female' and x['Pclass'] == 1:
        if x['Title'] == 1:
            return 30
        if x['Title'] == 2:
            return 40
        if x['Title'] == 3:
            return 41
        if x['Title'] == 5:
            return 49
    if x['Sex'] == 'female' and x['Pclass'] == 2:
        if x['Title'] == 1:
            return 24
        if x['Title'] == 2:
            return 32
    if x['Sex'] == 'female' and x['Pclass'] == 3:
         if x['Title'] == 1:
            return 18
         if x['Title'] == 2:
            return 31
    if x['Sex'] == 'male' and x['Pclass'] == 1:
        if x['Title'] == 0:
            return 40
        if x['Title'] == 3:
            return 40
        if x['Title'] == 4:
            return 4
        if x['Title'] == 5:
            return 51
    if x['Sex'] == 'male' and x['Pclass'] == 2:
        if x['Title'] == 0:
            return 31
        if x['Title'] == 4:
            return 1
        if x['Title'] == 5:
            return 47
    if x['Sex'] == 'male' and x['Pclass'] == 3:
        if x['Title'] == 0:
            return 26
        if x['Title'] == 4:
            return 4
def processAge(y):
    y['Age'] = y.apply(lambda x : processMissingAge(x) if np.isnan(x['Age']) else x['Age'],axis=1)

In [21]:
#procesSex
def processSex(y):
    sex = py.get_dummies(y['Sex'],drop_first=True)
    y['Embarked'].fillna('S',inplace=True)
    embark = py.get_dummies(y['Embarked'],drop_first=True)
    y.drop(['Embarked','Sex'],axis=1,inplace=True)
    y = py.concat([y,embark,sex],axis=1)

In [22]:
#processTicket
def processTicket(y):
    y.drop(['Ticket'],axis=1,inplace=True)

In [23]:
#process Fare
def processFare(y):
    y['Fare'] = y['Fare'].fillna(y['Fare'].mean())

In [24]:
#Do scaling on Age and fare
scale = StandardScaler()
def scaleAgeFare(y):
    std_scale = scale.fit(y[['Fare','Age']])
    y[['Fare','Age']] = scale.transform(y[['Fare','Age']])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('Survived',axis=1), data['Survived'], test_size=0.33, random_state=50)

In [25]:
from sklearn.pipeline import Pipeline

In [28]:
pipeline = Pipeline([('title',processTitle()),('NameLen',processNameLength()),('Age',processAge()),('Sex',processSex()),('Ticket',processTicket()),('Fare',processFare()),('scale',scaleAgeFare())])

TypeError: processTitle() missing 1 required positional argument: 'x'